In [2]:
from math import asin,sin,cos,pi,sqrt
from random import randrange
from time import time
import pygame

pygame.init()
pygame.font.init()

BLACK = (0,0,0)
WHITE = (255,255,255)

def fill(surface,color):
    w,h = surface.get_size()
    r,g,b = color
    for x in range(w):
        for y in range(h):
            a = surface.get_at((x,y))[3]
            surface.set_at((x,y),(r,g,b,a))

class Window:
    def __init__(self,screen_size,size):
        self.screen_size = screen_size
        self.size = size
        self.screen = pygame.display.set_mode(self.screen_size,pygame.RESIZABLE)
        pygame.display.set_caption("PONG")
        self.logo = pygame.image.load("yazgig_logo.png").convert_alpha()
        fill(self.logo,WHITE)
        pygame.display.set_icon(pygame.transform.scale(self.logo,(32,32)))
        fill(self.logo,(127,127,127))
        self.line_height = 12
        self.font_name = "arial.ttf"
        self.font = pygame.font.SysFont(self.font_name,int(6*pixel_ratio))
        self.background = self.get_background()
        self.origin = (0,0)
        self.input_file = open("inputs.txt","r")
        self.inputs = [0,0]
        self.scores = [0,0]
        self.running = True
        self.ball = Ball((self.screen_size[0]//2,self.screen_size[1]//2))
    def get_background(self):
        background = pygame.Surface(self.screen_size)
        background.fill(BLACK)
        pygame.draw.rect(background,WHITE,(0,0,int(self.size[0]*pixel_ratio),int(pixel_ratio))) # ceiling
        pygame.draw.rect(background,WHITE,(0,int((self.size[1]-1)*pixel_ratio),int(self.size[0]*pixel_ratio),int(pixel_ratio))) # floor
        logo = pygame.transform.scale(self.logo,(int(self.logo.get_width()*pixel_ratio/48),int(self.logo.get_height()*pixel_ratio/48)))
        background.blit(logo,(self.screen_size[0]//2-logo.get_width()//2,self.screen_size[1]//2-logo.get_height()//2))
        return background
    def new_round(self):
        self.ball = Ball((self.screen_size[0]//2,self.screen_size[1]//2))
    def update_inputs(self):
        self.input_file.seek(0)
        p1,p2 = self.input_file.read().split(",")
        # p1 = p2 = min(max(pygame.mouse.get_pos()[1]/(self.screen_size[1]-1),0),1)
        self.inputs[0] = int(float(p1)*(self.screen_size[1]-(2+self.line_height)*pixel_ratio)+pixel_ratio)
        self.inputs[1] = int(float(p2)*(self.screen_size[1]-(2+self.line_height)*pixel_ratio)+pixel_ratio)
    def draw(self):
        # background
        screen = self.background.copy()
        #
        for i in range(2):
            # scoreboard
            score_surface = self.font.render(str(self.scores[i]),False,WHITE)
            screen.blit(score_surface,(self.screen_size[0]*7//16+i*self.screen_size[0]*2//16-score_surface.get_width()//2,self.screen_size[1]//8-score_surface.get_height()//2))
            # sticks
            y = self.inputs[i]
            x = int((0 if i == 0 else self.size[0]-1)*pixel_ratio)
            w = pixel_ratio
            h = int(self.line_height*pixel_ratio)
            pygame.draw.rect(screen,WHITE,(x,y+w//2,w,h-w))
            pygame.draw.circle(screen,WHITE,(x+w//2,y+w//2),w//2)
            pygame.draw.circle(screen,WHITE,(x+w//2,y+h-w//2),w//2)
        self.ball.draw(screen)
        self.screen.blit(screen,self.origin)
        pygame.display.update()
    def resize(self,width,height):
        # self.screen_size = (int(height/9*16),height)
        # width = max(width,self.screen_size[0])
        if int(height/9*16) <= width:
            self.screen_size = (int(height/9*16),height)
        else:
            self.screen_size = (width,int(width/16*9))
        self.origin = ((width-self.screen_size[0])//2,(height-self.screen_size[1])//2)
        self.screen = pygame.display.set_mode((width,height),pygame.RESIZABLE)
        global pixel_ratio
        self.ball.speed /= pixel_ratio
        self.ball.pos[0] /= pixel_ratio
        self.ball.pos[1] /= pixel_ratio
        pixel_ratio = self.screen_size[1]/self.size[1]
        self.ball.pos[0] *= pixel_ratio
        self.ball.pos[1] *= pixel_ratio
        self.ball.speed *= pixel_ratio
        self.background = self.get_background()
        self.font = pygame.font.SysFont(self.font_name,int(6*pixel_ratio))
    def mainloop(self):
        clock = pygame.time.Clock()
        last_time = time()
        now = time()
        while self.running:
            clock.tick(60)
            now = time()
            delta_time = now-last_time
            last_time = now
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        self.running = False
                    if event.key == pygame.K_F5:
                        self.scores = [0,0]
                        self.new_round()
                elif event.type == pygame.VIDEORESIZE:
                    self.resize(event.w,event.h)
            self.update_inputs()
            self.ball.update(self,delta_time)
            self.draw()

class Ball:
    def __init__(self,pos):
        self.pos = list(pos)
        angle = randrange(2)*pi
        self.vel = [cos(angle),sin(angle)]
        self.speed = 30*pixel_ratio
    def update(self,window,delta_time):
        pos = [
            self.pos[0]+self.vel[0]*self.speed*delta_time,
            self.pos[1]+self.vel[1]*self.speed*delta_time,
        ]
        if not (pixel_ratio <= pos[1] < (window.size[1]-1)*pixel_ratio): # if ball is above or below
            self.vel[1] *= -1
        if not (pixel_ratio <= pos[0] < (window.size[0]-1)*pixel_ratio): # if ball is behind the sticks
            player = 0 if pos[0] <= (window.size[0]//2*pixel_ratio) else 1
            y1 = window.inputs[player]
            y2 = y1+window.line_height*pixel_ratio
            if y1 <= pos[1] <= y2: # if ball hits a stick
                y_mid = (y1+y2)/2
                ratio = (pos[1]-y_mid)/(y_mid-y1)/sqrt(2)
                new_angle = asin(ratio) # between -pi/4 and pi/4
                new_angle = round(new_angle/(pi/180))*(pi/180)
                dir = 1 if player == 0 else -1
                self.vel = [cos(new_angle)*dir,sin(new_angle)]
                self.speed = min(self.speed+2*pixel_ratio,200*pixel_ratio)
            else:
                window.new_round()
                window.scores[(player+1)%2] += 1
        self.pos[0] += self.vel[0]*self.speed*delta_time
        self.pos[1] += self.vel[1]*self.speed*delta_time
    def draw(self,screen):
        pygame.draw.circle(screen,WHITE,(int(self.pos[0]),int(self.pos[1])),pixel_ratio//2)

if __name__ == "__main__":
    w = pygame.display.Info().current_w//2
    screen_size = (w,int(w/16*9))
    size = (80,45)
    pixel_ratio = screen_size[1]/size[1]
    window = Window(screen_size,size)
    window.mainloop()